<a href="https://www.kaggle.com/code/averma111/lightning-timeseries?scriptVersionId=136706980" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [171]:
# Re-loads all imports every time the cell is ran. 
%reload_ext autoreload

from time import time

import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,.5f}'.format

from IPython.display import display

# Sklearn tools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Neural Networks
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers.csv_logs import CSVLogger

# Plotting
%matplotlib inline
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/playground-series-s3e19/sample_submission.csv
/kaggle/input/playground-series-s3e19/train.csv
/kaggle/input/playground-series-s3e19/test.csv


In [172]:
class TimeseriesDataset(Dataset):   
    def __init__(self, X: np.ndarray, y: np.ndarray, seq_len: int = 1):
        self.X = torch.tensor(X).float()
        self.y = torch.tensor(y).float()
        self.seq_len = seq_len

    def __len__(self):
        return self.X.__len__() - (self.seq_len-1)

    def __getitem__(self, index):
        return (self.X[index:index+self.seq_len], self.y[index+self.seq_len-1])

In [173]:
from sklearn.preprocessing import LabelEncoder
class ForecastingDataModule(pl.LightningDataModule):
    
    def __init__(self, seq_len = 1, batch_size = 128, num_workers=0):
        super().__init__()
        self.seq_len = seq_len
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.X_train = None
        self.y_train = None
        self.X_val = None
        self.y_val = None
        self.X_test = None
        self.X_test = None?
        self.columns = None
        self.preprocessing = None

    def prepare_data(self):
        path = '/kaggle/input/playground-series-s3e19/train.csv'
        df = pd.read_csv(
            path, 
            sep=',', 
            parse_dates=['date'], 
            infer_datetime_format=True, 
            low_memory=False
        )

        X = df.copy()
        X['store'] =  LabelEncoder().fit_transform(X['store'])
        X['product'] =  LabelEncoder().fit_transform(X['product'])
        X['country'] =  LabelEncoder().fit_transform(X['country'])
        X = X.loc[:, X.columns!='id']
        y = X['num_sold']
        X.drop(columns=['date','num_sold'],axis=1,inplace=True)
        self.columns = X.columns
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.25, shuffle=False
        )
        #print(X_train.shape,X_val.shape,y_train.shape,y_val.shape)
        self.X_train=X_train
        self.X_val = X_val
        self.y_train = y_train
        self.y_val = y_val

    def setup(self, stage=None):
        preprocessing = StandardScaler()
        if stage == 'fit':
            self.X_train = preprocessing.fit_transform(self.X_train)
            self.y_train = self.y_train.values
            self.X_val = preprocessing.fit_transform(self.X_val)
            self.y_val = self.y_val.values

        if stage == 'test':
            self.X_test = preprocessing.fit_transform(X_test)
            
        

    def train_dataloader(self):
        train_dataset = TimeseriesDataset(self.X_train, 
                                          self.y_train, 
                                          seq_len=self.seq_len)
        train_loader = DataLoader(train_dataset, 
                                  batch_size = self.batch_size, 
                                  shuffle = False, 
                                  num_workers = self.num_workers)
        
        return train_loader

    def val_dataloader(self):
        val_dataset = TimeseriesDataset(self.X_val, 
                                        self.y_val, 
                                        seq_len=self.seq_len)
        val_loader = DataLoader(val_dataset, 
                                batch_size = self.batch_size, 
                                shuffle = False, 
                                num_workers = self.num_workers)

        return val_loader

    def test_dataloader(self):
        test_dataset = TimeseriesDataset(self.X_test, 
                                         self.y_test, 
                                         seq_len=self.seq_len)
        test_loader = DataLoader(test_dataset, 
                                 batch_size = self.batch_size, 
                                 shuffle = False, 
                                 num_workers = self.num_workers)

        return test_loader

In [174]:
class LSTMRegressor(pl.LightningModule):
    def __init__(self, 
                 n_features, 
                 hidden_size, 
                 seq_len, 
                 batch_size,
                 num_layers, 
                 dropout, 
                 learning_rate,
                 criterion):
        super(LSTMRegressor, self).__init__()
        self.n_features = n_features
        self.hidden_size = hidden_size
        self.seq_len = seq_len
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.dropout = dropout
        self.criterion = criterion
        self.learning_rate = learning_rate

        self.lstm = nn.LSTM(input_size=n_features, 
                            hidden_size=hidden_size,
                            num_layers=num_layers, 
                            dropout=dropout, 
                            batch_first=True)
        self.linear = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        y_pred = self.linear(lstm_out[:,-1])
        return y_pred
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        result = pl.TrainResult(loss)
        result.log('Train_loss', loss)
        return result

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        result = pl.EvalResult(checkpoint_on=loss)
        result.log('Validation_loss', loss)
        return result
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        result = pl.EvalResult()
        result.log('Test_loss', loss)
        return result

In [175]:
p = dict(
    seq_len = 24,
    batch_size = 70, 
    criterion = nn.MSELoss(),
    max_epochs = 10,
    n_features = 3,
    hidden_size = 100,
    num_layers = 1,
    dropout = 0.2,
    learning_rate = 0.001,
)

In [176]:
seed_everything(1)
csv_logger = CSVLogger('./', name='lstm', version='0'),

model = LSTMRegressor(
    n_features = p['n_features'],
    hidden_size = p['hidden_size'],
    seq_len = p['seq_len'],
    batch_size = p['batch_size'],
    criterion = p['criterion'],
    num_layers = p['num_layers'],
    dropout = p['dropout'],
    learning_rate = p['learning_rate']
)



In [177]:
print(model)

LSTMRegressor(
  (criterion): MSELoss()
  (lstm): LSTM(3, 100, batch_first=True, dropout=0.2)
  (linear): Linear(in_features=100, out_features=1, bias=True)
)


In [178]:
trainer = Trainer(
    max_epochs=p['max_epochs'],
    logger=csv_logger,
    accelerator='auto',
    log_every_n_steps=1
)

In [179]:
dm = ForecastingDataModule(
    seq_len = p['seq_len'],
    batch_size = p['batch_size']
)
dm.prepare_data()


Type:        NoneType
String form: None
Namespace:   Python builtin
Docstring:   <no docstring>

In [182]:
trainer.fit(model, dm)
#trainer.test(model, datamodule=dm)

(102712, 3) (102712,) (34238, 3) (34238,)


Sanity Checking: 0it [00:00, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 trainer.fit(model, dm)                                                                       │
│   2 #trainer.test(model, datamodule=dm)                                                          │
│   3                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:531 in fit          │
│                                                                                                  │
│    528 │   │   """                                                                               │
│    529 │   │   model = _maybe_unwrap_optimized(model)                                            │
│    530 │   │   self.strategy._lightning_module = model                                           │
│ ❱  531 │   │   call._call_and_handle_interrupt(                                                  │
│    532 │   │   │   self, self._fit_impl, model, train_dataloaders, val_dataloaders, datamodule,  │
│    533 │   │   )                                                                                 │
│    534                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:42 in                  │
│ _call_and_handle_interrupt                                                                       │
│                                                                                                  │
│    39 │   try:                                                                                   │
│    40 │   │   if trainer.strategy.launcher is not None:                                          │
│    41 │   │   │   return trainer.strategy.launcher.launch(trainer_fn, *args, trainer=trainer,    │
│ ❱  42 │   │   return trainer_fn(*args, **kwargs)                                                 │
│    43 │                                                                                          │
│    44 │   except _TunerExitException:                                                            │
│    45 │   │   _call_teardown_hook(trainer)                                                       │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:570 in _fit_impl    │
│                                                                                                  │
│    567 │   │   │   model_provided=True,                                                          │
│    568 │   │   │   model_connected=self.lightning_module is not None,                            │
│    569 │   │   )                                                                                 │
│ ❱  570 │   │   self._run(model, ckpt_path=ckpt_path)                                             │
│    571 │   │                                                                                     │
│    572 │   │   assert self.state.stopped                                                         │
│    573 │   │   self.training = False                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:975 in _run         │
│                                                            

In [181]:
metrics = pd.read_csv('./lstm/0/metrics.csv')
train_loss = metrics[['Train_loss', 'step', 'epoch']][~np.isnan(metrics['Train_loss'])]
val_loss = metrics[['Val_loss', 'epoch']][~np.isnan(metrics['Val_loss'])]
#test_loss = metrics['Test_loss'].iloc[-1]

fig, axes = plt.subplots(1, 2, figsize=(16, 5), dpi=100)
axes[0].set_title('Train loss per batch')
axes[0].plot(train_loss['step'], train_loss['train_loss'])
axes[1].set_title('Validation loss per epoch')
axes[1].plot(val_loss['epoch'], val_loss['Val_loss'], color='orange')
plt.show(block = True)

print('MSE:')
print(f"Train loss: {train_loss['Train_loss'].iloc[-1]:.3f}")
print(f"Val loss:   {val_loss['Val_loss'].iloc[-1]:.3f}")
#print(f'Test loss:  {test_loss:.3f}')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 metrics = pd.read_csv('./lstm/0/metrics.csv')                                               │
│    2 train_loss = metrics[['Train_loss', 'step', 'epoch']][~np.isnan(metrics['Train_loss'])]     │
│    3 val_loss = metrics[['Val_loss', 'epoch']][~np.isnan(metrics['Val_loss'])]                   │
│    4 #test_loss = metrics['Test_loss'].iloc[-1]                                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/pandas/util/_decorators.py:211 in wrapper                │
│                                                                                                  │
│   208 │   │   │   │   │   raise TypeError(msg)                                                   │
│   209 │   │   │   │   else:                                                                      │
│   210 │   │   │   │   │   kwargs[new_arg_name] = new_arg_value                                   │
│ ❱ 211 │   │   │   return func(*args, **kwargs)                                                   │
│   212 │   │                                                                                      │
│   213 │   │   return cast(F, wrapper)                                                            │
│   214                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/pandas/util/_decorators.py:331 in wrapper                │
│                                                                                                  │
│   328 │   │   │   │   │   FutureWarning,                                                         │
│   329 │   │   │   │   │   stacklevel=find_stack_level(),                                         │
│   330 │   │   │   │   )                                                                          │
│ ❱ 331 │   │   │   return func(*args, **kwargs)                                                   │
│   332 │   │                                                                                      │
│   333 │   │   # error: "Callable[[VarArg(Any), KwArg(Any)], Any]" has no                         │
│   334 │   │   # attribute "__signature__"                                                        │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/pandas/io/parsers/readers.py:950 in read_csv             │
│                                                                                                  │
│    947 │   )                                                                                     │
│    948 │   kwds.update(kwds_defaults)                                                            │
│    949 │                                                                                         │
│ ❱  950 │   return _read(filepath_or_buffer, kwds)                                                │
│    951                                                                                           │
│    952                                                                                           │
│    953 # iterator=True -> TextFileReader                                                         │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/pandas/io/parsers/readers.py:605 in _read                │
│                                                            